# Experimentos

In [1]:
# imports
import numpy as np
import pandas as pd

# time
import time
from datetime import timedelta

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# ETL
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Oversampling Methods
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# Deep Learning Framework - Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.utils import to_categorical
from keras.backend import clear_session
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

# K-fold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

models = pd.DataFrame()

Using TensorFlow backend.


#  Raw Data

In [2]:
## Data
df = pd.read_csv('data/train.csv')

## Remove duplicate rows
df.drop_duplicates(inplace=True)

# Slit (X,Y)
x_data = df.drop(['ID','TARGET'], axis=1)
y_data = df['TARGET'].copy()

x_data.shape

(76020, 369)

# Oversampling: Random Oversampling

## CNN 1 MAX RO

In [3]:
# info
model_name = 'CNN 1 MAX RO'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)

    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97584 samples, validate on 13684 samples
Epoch 1/10000
97584/97584 [==============================] - 29s - loss: 0.5497 - acc: 0.7249 - val_loss: 0.4651 - val_acc: 0.7554
Epoch 2/10000
97584/97584 [==============================] - 14s - loss: 0.4978 - acc: 0.7580 - val_loss: 0.4594 - val_acc: 0.7803
Epoch 3/10000
97584/97584 [==============================] - 13s - loss: 0.4743 - acc: 0.7727 - val_loss: 0.4849 - val_acc: 0.7669
Epoch 4/10000
97584/97584 [==============================] - 13s - loss: 0.4560 - acc: 0.7855 - val_loss: 0.4997 - val_acc: 0.7835
Epoch 5/10000
97584/97584 [==============================] - 13s - loss: 0.4430 - acc: 0.7933 - val_loss: 0.4585 - val_acc: 0.7865
Epoch 6/10000
97584/97584 [==============================] - 13s - loss: 0.4312 - acc: 0.8024 - val_loss: 0.6081 - val_acc: 0.6828
Epoch 7/10000
97584/97584 [==============================] - 13s - loss: 0.4197 - acc: 0.8078 - val_loss: 0.4864 - val_acc: 0.7800
Epoch 8/10000
97584/97584 [=

97554/97554 [==============================] - 12s - loss: 0.3642 - acc: 0.8471 - val_loss: 0.4270 - val_acc: 0.8300
Epoch 24/10000
97554/97554 [==============================] - 12s - loss: 0.3587 - acc: 0.8496 - val_loss: 0.4371 - val_acc: 0.8316
Epoch 25/10000
97554/97554 [==============================] - 12s - loss: 0.3538 - acc: 0.8519 - val_loss: 0.3764 - val_acc: 0.8690
Epoch 26/10000
97554/97554 [==============================] - 12s - loss: 0.3507 - acc: 0.8529 - val_loss: 0.4738 - val_acc: 0.8270
Epoch 27/10000
97554/97554 [==============================] - 12s - loss: 0.3506 - acc: 0.8534 - val_loss: 0.4541 - val_acc: 0.8118
Epoch 28/10000
97554/97554 [==============================] - 12s - loss: 0.3465 - acc: 0.8558 - val_loss: 0.4507 - val_acc: 0.8113
Epoch 29/10000
97554/97554 [==============================] - 12s - loss: 0.3435 - acc: 0.8580 - val_loss: 0.5159 - val_acc: 0.7998
Epoch 30/10000
97554/97554 [==============================] - 12s - loss: 0.3433 - acc: 0.8

97550/97550 [==============================] - 13s - loss: 0.3527 - acc: 0.8444 - val_loss: 0.4716 - val_acc: 0.7818
Epoch 15/10000
97550/97550 [==============================] - 12s - loss: 0.3474 - acc: 0.8473 - val_loss: 0.4400 - val_acc: 0.8102
Epoch 16/10000
97550/97550 [==============================] - 13s - loss: 0.3464 - acc: 0.8481 - val_loss: 0.5197 - val_acc: 0.7847
Epoch 17/10000
97550/97550 [==============================] - 12s - loss: 0.3458 - acc: 0.8494 - val_loss: 0.4827 - val_acc: 0.7982
Epoch 18/10000
97550/97550 [==============================] - 12s - loss: 0.3386 - acc: 0.8536 - val_loss: 0.5155 - val_acc: 0.7947
Epoch 19/10000
97550/97550 [==============================] - 13s - loss: 0.3288 - acc: 0.8561 - val_loss: 0.4474 - val_acc: 0.8175
Epoch 20/10000
97550/97550 [==============================] - 12s - loss: 0.3242 - acc: 0.8577 - val_loss: 0.5172 - val_acc: 0.7814
Epoch 21/10000
97550/97550 [==============================] - 12s - loss: 0.3239 - acc: 0.8

97598/97598 [==============================] - 13s - loss: 0.3028 - acc: 0.8653 - val_loss: 0.4616 - val_acc: 0.8315
Epoch 31/10000
97598/97598 [==============================] - 13s - loss: 0.2997 - acc: 0.8656 - val_loss: 0.4542 - val_acc: 0.8326
Epoch 32/10000
97598/97598 [==============================] - 12s - loss: 0.2951 - acc: 0.8674 - val_loss: 0.4825 - val_acc: 0.8228
Epoch 33/10000
97598/97598 [==============================] - 12s - loss: 0.2932 - acc: 0.8693 - val_loss: 0.4547 - val_acc: 0.8243
Epoch 34/10000
97598/97598 [==============================] - 12s - loss: 0.2977 - acc: 0.8676 - val_loss: 0.5390 - val_acc: 0.7896
Epoch 35/10000
97598/97598 [==============================] - 12s - loss: 0.2922 - acc: 0.8698 - val_loss: 0.4869 - val_acc: 0.8147
Epoch 36/10000
97598/97598 [==============================] - 12s - loss: 0.2905 - acc: 0.8706 - val_loss: 0.4906 - val_acc: 0.7979
Epoch 37/10000
97598/97598 [==============================] - 14s - loss: 0.2899 - acc: 0.8

In [4]:
print(model_name)

metrics_kf

CNN 1 MAX RO


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.947677,0.712376,0.712376,0.545603,0.691750,0.538749,0.805603,00:07:17.280199
KF_2,0.947724,0.735170,0.735170,0.551326,0.714851,0.547738,0.810206,00:07:07.804183
KF_3,0.947996,0.726688,0.726688,0.544195,0.703711,0.529227,0.782689,00:07:49.189549
KF_4,0.945987,0.748324,0.748324,0.551952,0.702986,0.552886,0.824125,00:07:57.106923
KF_5,0.938796,0.727555,0.727555,0.555252,0.698600,0.561817,0.840174,00:06:34.343148
KF_6,0.953887,0.738962,0.738962,0.560297,0.716170,0.569482,0.843331,00:13:05.791750
KF_7,0.949969,0.767989,0.771311,0.555254,0.735063,0.552567,0.809261,00:07:12.028757
KF_8,0.948837,0.714068,0.714068,0.547159,0.674626,0.548367,0.830834,00:08:39.895776
KF_9,0.943787,0.750788,0.750788,0.556458,0.721290,0.559672,0.826865,00:07:15.471211
KF_10,0.952550,0.713250,0.713250,0.553626,0.700673,0.557732,0.833311,00:09:11.012606


## CNN 2 MAX RO

In [5]:
# info
model_name = 'CNN 2 MAX RO'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(96, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()
    
metrics_kf

KF_1
Train on 97588 samples, validate on 13684 samples
Epoch 1/10000
97588/97588 [==============================] - 14s - loss: 0.5963 - acc: 0.6958 - val_loss: 0.5937 - val_acc: 0.7100
Epoch 2/10000
97588/97588 [==============================] - 13s - loss: 0.5350 - acc: 0.7382 - val_loss: 0.5540 - val_acc: 0.7195
Epoch 3/10000
97588/97588 [==============================] - 12s - loss: 0.5175 - acc: 0.7459 - val_loss: 0.5263 - val_acc: 0.7224
Epoch 4/10000
97588/97588 [==============================] - 13s - loss: 0.5058 - acc: 0.7551 - val_loss: 0.4858 - val_acc: 0.7711
Epoch 5/10000
97588/97588 [==============================] - 13s - loss: 0.4927 - acc: 0.7612 - val_loss: 0.4188 - val_acc: 0.8073
Epoch 6/10000
97588/97588 [==============================] - 13s - loss: 0.4818 - acc: 0.7681 - val_loss: 0.4131 - val_acc: 0.8210
Epoch 7/10000
97588/97588 [==============================] - 13s - loss: 0.4700 - acc: 0.7747 - val_loss: 0.4930 - val_acc: 0.7697
Epoch 8/10000
97588/97588 [=

97536/97536 [==============================] - 13s - loss: 0.4083 - acc: 0.8129 - val_loss: 0.5246 - val_acc: 0.7464
Epoch 22/10000
97536/97536 [==============================] - 12s - loss: 0.3892 - acc: 0.8219 - val_loss: 0.4685 - val_acc: 0.7978
Epoch 23/10000
97536/97536 [==============================] - 12s - loss: 0.3836 - acc: 0.8247 - val_loss: 0.4952 - val_acc: 0.7781
Epoch 24/10000
97536/97536 [==============================] - 12s - loss: 0.3854 - acc: 0.8251 - val_loss: 0.4728 - val_acc: 0.7745
Epoch 25/10000
97536/97536 [==============================] - 12s - loss: 0.3820 - acc: 0.8277 - val_loss: 0.4630 - val_acc: 0.7992
Epoch 26/10000
97536/97536 [==============================] - 12s - loss: 0.3774 - acc: 0.8298 - val_loss: 0.5642 - val_acc: 0.7265
Epoch 27/10000
97536/97536 [==============================] - 12s - loss: 0.3723 - acc: 0.8328 - val_loss: 0.4839 - val_acc: 0.7801
Epoch 28/10000
97536/97536 [==============================] - 13s - loss: 0.3706 - acc: 0.8

97344/97572 [============================>.] - ETA: 0sKF_8
Train on 97670 samples, validate on 13684 samples
Epoch 1/10000
97670/97670 [==============================] - 14s - loss: 0.5884 - acc: 0.6976 - val_loss: 0.6274 - val_acc: 0.6656
Epoch 2/10000
97670/97670 [==============================] - 13s - loss: 0.5360 - acc: 0.7336 - val_loss: 0.6866 - val_acc: 0.6785
Epoch 3/10000
97670/97670 [==============================] - 13s - loss: 0.5221 - acc: 0.7405 - val_loss: 0.3460 - val_acc: 0.8507
Epoch 4/10000
97670/97670 [==============================] - 13s - loss: 0.5137 - acc: 0.7462 - val_loss: 0.5317 - val_acc: 0.7478
Epoch 5/10000
97670/97670 [==============================] - 13s - loss: 0.4994 - acc: 0.7546 - val_loss: 0.6544 - val_acc: 0.6291
Epoch 6/10000
97670/97670 [==============================] - 13s - loss: 0.4901 - acc: 0.7600 - val_loss: 0.4603 - val_acc: 0.7701
Epoch 7/10000
97670/97670 [==============================] - 13s - loss: 0.4812 - acc: 0.7656 - val_loss:

,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.921459,0.744826,0.744826,0.544306,0.710560,0.526428,0.774431,00:06:55.634692
KF_2,0.918912,0.785753,0.785753,0.546866,0.739557,0.521799,0.753650,00:07:28.674628
KF_3,0.928980,0.753734,0.753734,0.543305,0.705561,0.525020,0.774007,00:09:22.472864
KF_4,0.934336,0.777048,0.780370,0.554782,0.729994,0.552772,0.811760,00:08:46.059467
KF_5,0.925689,0.740167,0.740167,0.544372,0.695403,0.533425,0.794265,00:07:04.070707
KF_6,0.932217,0.763081,0.763081,0.548926,0.712819,0.541311,0.800184,00:07:53.819763
KF_7,0.933630,0.782696,0.782696,0.548406,0.728799,0.533192,0.778874,00:07:38.240762
KF_8,0.910607,0.747926,0.751249,0.547731,0.712648,0.537583,0.793738,00:05:20.956879
KF_9,0.919801,0.760204,0.760204,0.545122,0.722596,0.523884,0.764899,00:07:00.814942
KF_10,0.933810,0.725864,0.729197,0.550260,0.716040,0.544427,0.804499,00:08:06.207301


In [6]:
print(model_name)

metrics_kf

CNN 2 MAX RO


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.921459,0.744826,0.744826,0.544306,0.710560,0.526428,0.774431,00:06:55.634692
KF_2,0.918912,0.785753,0.785753,0.546866,0.739557,0.521799,0.753650,00:07:28.674628
KF_3,0.928980,0.753734,0.753734,0.543305,0.705561,0.525020,0.774007,00:09:22.472864
KF_4,0.934336,0.777048,0.780370,0.554782,0.729994,0.552772,0.811760,00:08:46.059467
KF_5,0.925689,0.740167,0.740167,0.544372,0.695403,0.533425,0.794265,00:07:04.070707
KF_6,0.932217,0.763081,0.763081,0.548926,0.712819,0.541311,0.800184,00:07:53.819763
KF_7,0.933630,0.782696,0.782696,0.548406,0.728799,0.533192,0.778874,00:07:38.240762
KF_8,0.910607,0.747926,0.751249,0.547731,0.712648,0.537583,0.793738,00:05:20.956879
KF_9,0.919801,0.760204,0.760204,0.545122,0.722596,0.523884,0.764899,00:07:00.814942
KF_10,0.933810,0.725864,0.729197,0.550260,0.716040,0.544427,0.804499,00:08:06.207301


## CNN 3 MAX RO

In [7]:
# info
model_name = 'CNN 3 MAX RO'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(24, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97464 samples, validate on 13684 samples
Epoch 1/10000
97464/97464 [==============================] - 14s - loss: 0.6393 - acc: 0.6506 - val_loss: 0.6194 - val_acc: 0.5479
Epoch 2/10000
97464/97464 [==============================] - 14s - loss: 0.5766 - acc: 0.7017 - val_loss: 0.5431 - val_acc: 0.6473
Epoch 3/10000
97464/97464 [==============================] - 14s - loss: 0.5639 - acc: 0.7121 - val_loss: 0.5343 - val_acc: 0.6481
Epoch 4/10000
97464/97464 [==============================] - 14s - loss: 0.5581 - acc: 0.7157 - val_loss: 0.5093 - val_acc: 0.6813
Epoch 5/10000
97464/97464 [==============================] - 14s - loss: 0.5551 - acc: 0.7167 - val_loss: 0.4929 - val_acc: 0.6743
Epoch 6/10000
97464/97464 [==============================] - 14s - loss: 0.5516 - acc: 0.7184 - val_loss: 0.6117 - val_acc: 0.5883
Epoch 7/10000
97464/97464 [==============================] - 14s - loss: 0.5466 - acc: 0.7224 - val_loss: 0.5688 - val_acc: 0.6290
Epoch 8/10000
97464/97464 [=

97494/97494 [==============================] - 14s - loss: 0.5306 - acc: 0.7314 - val_loss: 0.4560 - val_acc: 0.7367
Epoch 15/10000
97494/97494 [==============================] - 14s - loss: 0.5286 - acc: 0.7329 - val_loss: 0.5155 - val_acc: 0.6986
Epoch 16/10000
97494/97494 [==============================] - 14s - loss: 0.5257 - acc: 0.7330 - val_loss: 0.4609 - val_acc: 0.7015
Epoch 17/10000
97494/97494 [==============================] - 14s - loss: 0.5243 - acc: 0.7344 - val_loss: 0.5387 - val_acc: 0.6646
Epoch 18/10000
97494/97494 [==============================] - 14s - loss: 0.5246 - acc: 0.7344 - val_loss: 0.5483 - val_acc: 0.6588
Epoch 19/10000
97494/97494 [==============================] - 14s - loss: 0.5253 - acc: 0.7344 - val_loss: 0.5384 - val_acc: 0.6378
Epoch 20/10000
97494/97494 [==============================] - 14s - loss: 0.5241 - acc: 0.7346 - val_loss: 0.5259 - val_acc: 0.7026
Epoch 21/10000
97494/97494 [==============================] - 14s - loss: 0.5214 - acc: 0.7

97534/97534 [==============================] - 13s - loss: 0.5279 - acc: 0.7459 - val_loss: 0.5200 - val_acc: 0.7886
Epoch 15/10000
97534/97534 [==============================] - 14s - loss: 0.5268 - acc: 0.7467 - val_loss: 0.5666 - val_acc: 0.7516
Epoch 16/10000
97534/97534 [==============================] - 13s - loss: 0.5232 - acc: 0.7498 - val_loss: 0.5437 - val_acc: 0.7632
Epoch 17/10000
97534/97534 [==============================] - 14s - loss: 0.5230 - acc: 0.7495 - val_loss: 0.4362 - val_acc: 0.8120
Epoch 18/10000
97534/97534 [==============================] - 13s - loss: 0.5231 - acc: 0.7503 - val_loss: 0.5418 - val_acc: 0.7618
Epoch 19/10000
97534/97534 [==============================] - 14s - loss: 0.5215 - acc: 0.7505 - val_loss: 0.6089 - val_acc: 0.6999
Epoch 20/10000
97534/97534 [==============================] - 13s - loss: 0.5231 - acc: 0.7518 - val_loss: 0.5187 - val_acc: 0.7946
Epoch 21/10000
97534/97534 [==============================] - 14s - loss: 0.5217 - acc: 0.7

97408/97408 [==============================] - 15s - loss: 0.5074 - acc: 0.7539 - val_loss: 0.4603 - val_acc: 0.7283
Epoch 59/10000
97408/97408 [==============================] - 14s - loss: 0.5077 - acc: 0.7557 - val_loss: 0.5209 - val_acc: 0.6946
Epoch 60/10000
97408/97408 [==============================] - 14s - loss: 0.5076 - acc: 0.7532 - val_loss: 0.6074 - val_acc: 0.6105
Epoch 61/10000
97408/97408 [==============================] - 15s - loss: 0.5074 - acc: 0.7533 - val_loss: 0.4641 - val_acc: 0.7379
Epoch 62/10000
97088/97408 [============================>.] - ETA: 0sKF_7
Train on 97592 samples, validate on 13684 samples
Epoch 1/10000
97592/97592 [==============================] - 15s - loss: 0.6549 - acc: 0.6400 - val_loss: 0.6478 - val_acc: 0.7021
Epoch 2/10000
97592/97592 [==============================] - 14s - loss: 0.6341 - acc: 0.6661 - val_loss: 0.5863 - val_acc: 0.7120
Epoch 3/10000
97592/97592 [==============================] - 14s - loss: 0.6155 - acc: 0.6773 - val_l

97426/97426 [==============================] - 14s - loss: 0.5361 - acc: 0.7375 - val_loss: 0.5387 - val_acc: 0.6789
Epoch 24/10000
97426/97426 [==============================] - 14s - loss: 0.5335 - acc: 0.7400 - val_loss: 0.5516 - val_acc: 0.6549
Epoch 25/10000
97426/97426 [==============================] - 14s - loss: 0.5349 - acc: 0.7385 - val_loss: 0.6263 - val_acc: 0.5982
Epoch 26/10000
97426/97426 [==============================] - 14s - loss: 0.5351 - acc: 0.7386 - val_loss: 0.5546 - val_acc: 0.6625
Epoch 27/10000
97426/97426 [==============================] - 14s - loss: 0.5308 - acc: 0.7413 - val_loss: 0.5326 - val_acc: 0.6671
Epoch 28/10000
97426/97426 [==============================] - 13s - loss: 0.5282 - acc: 0.7442 - val_loss: 0.5340 - val_acc: 0.6923
Epoch 29/10000
97426/97426 [==============================] - 14s - loss: 0.5279 - acc: 0.7440 - val_loss: 0.5962 - val_acc: 0.6340
Epoch 30/10000
97426/97426 [==============================] - 14s - loss: 0.5304 - acc: 0.7

In [8]:
print(model_name)

metrics_kf

CNN 3 MAX RO


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.834608,0.794271,0.794271,0.540732,0.723894,0.501408,0.720505,00:08:15.761711
KF_2,0.830486,0.809068,0.809068,0.546628,0.748791,0.515222,0.737735,00:17:40.329859
KF_3,0.842185,0.792122,0.792122,0.538221,0.719580,0.488880,0.696922,00:14:21.926545
KF_4,0.845695,0.818811,0.818811,0.554853,0.749279,0.546699,0.793738,00:08:55.557145
KF_5,0.841015,0.779277,0.779277,0.534035,0.699939,0.477458,0.680610,00:09:22.721826
KF_6,0.848474,0.802213,0.802213,0.535126,0.716469,0.464563,0.648119,00:15:11.848542
KF_7,0.766776,0.746095,0.749417,0.534776,0.702045,0.481237,0.687714,00:06:30.027122
KF_8,0.780012,0.723774,0.727097,0.528136,0.680976,0.433828,0.595238,00:08:42.056668
KF_9,0.836887,0.792849,0.792849,0.532531,0.708863,0.443015,0.606499,00:07:29.893614
KF_10,0.830739,0.784794,0.784794,0.541267,0.723991,0.504835,0.727667,00:08:21.815568


# Oversampling: SMOTE

## CNN 1 MAX SM

In [9]:
# info
model_name = 'CNN 1 MAX SM'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97572 samples, validate on 13684 samples
Epoch 1/10000
97572/97572 [==============================] - 14s - loss: 0.4053 - acc: 0.8281 - val_loss: 0.5047 - val_acc: 0.7949
Epoch 2/10000
97572/97572 [==============================] - 12s - loss: 0.3274 - acc: 0.8661 - val_loss: 0.3870 - val_acc: 0.8340
Epoch 3/10000
97572/97572 [==============================] - 12s - loss: 0.3052 - acc: 0.8772 - val_loss: 0.2888 - val_acc: 0.8705
Epoch 4/10000
97572/97572 [==============================] - 13s - loss: 0.2899 - acc: 0.8843 - val_loss: 0.2254 - val_acc: 0.9087
Epoch 5/10000
97572/97572 [==============================] - 13s - loss: 0.2767 - acc: 0.8893 - val_loss: 0.4109 - val_acc: 0.8284
Epoch 6/10000
97572/97572 [==============================] - 12s - loss: 0.2688 - acc: 0.8937 - val_loss: 0.2817 - val_acc: 0.8774
Epoch 7/10000
97572/97572 [==============================] - 13s - loss: 0.2600 - acc: 0.8967 - val_loss: 0.3035 - val_acc: 0.8579
Epoch 8/10000
97572/97572 [=

97582/97582 [==============================] - 12s - loss: 0.2279 - acc: 0.9107 - val_loss: 0.3145 - val_acc: 0.8693
Epoch 22/10000
97582/97582 [==============================] - 12s - loss: 0.2245 - acc: 0.9111 - val_loss: 0.3506 - val_acc: 0.8506
Epoch 23/10000
97582/97582 [==============================] - 12s - loss: 0.2247 - acc: 0.9122 - val_loss: 0.3060 - val_acc: 0.8759
Epoch 24/10000
97582/97582 [==============================] - 12s - loss: 0.2226 - acc: 0.9124 - val_loss: 0.2827 - val_acc: 0.8911
Epoch 25/10000
97582/97582 [==============================] - 12s - loss: 0.2224 - acc: 0.9133 - val_loss: 0.3031 - val_acc: 0.8768
Epoch 26/10000
97312/97582 [============================>.] - ETA: 0sKF_6
Train on 97586 samples, validate on 13684 samples
Epoch 1/10000
97586/97586 [==============================] - 13s - loss: 0.4057 - acc: 0.8265 - val_loss: 0.4119 - val_acc: 0.8320
Epoch 2/10000
97586/97586 [==============================] - 12s - loss: 0.3278 - acc: 0.8662 - val_

97397/97397 [==============================] - 14s - loss: 0.2432 - acc: 0.9049 - val_loss: 0.3300 - val_acc: 0.8633
Epoch 16/10000
97397/97397 [==============================] - 13s - loss: 0.2406 - acc: 0.9045 - val_loss: 0.3498 - val_acc: 0.8519
Epoch 17/10000
97397/97397 [==============================] - 14s - loss: 0.2389 - acc: 0.9059 - val_loss: 0.4060 - val_acc: 0.8365
Epoch 18/10000
97397/97397 [==============================] - 14s - loss: 0.2376 - acc: 0.9068 - val_loss: 0.2963 - val_acc: 0.8705
Epoch 19/10000
97397/97397 [==============================] - 15s - loss: 0.2352 - acc: 0.9069 - val_loss: 0.3476 - val_acc: 0.8635
Epoch 20/10000
97397/97397 [==============================] - 14s - loss: 0.2328 - acc: 0.9079 - val_loss: 0.3378 - val_acc: 0.8673
Epoch 21/10000
97397/97397 [==============================] - 14s - loss: 0.2301 - acc: 0.9095 - val_loss: 0.3997 - val_acc: 0.8242
Epoch 22/10000
97397/97397 [==============================] - 14s - loss: 0.2291 - acc: 0.9

In [10]:
print(model_name)

metrics_kf

CNN 1 MAX SM


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.973847,0.765979,0.769301,0.561991,0.679868,0.577579,0.871498,00:05:39.307981
KF_2,0.970315,0.783093,0.786415,0.566836,0.725665,0.581347,0.855452,00:05:01.790801
KF_3,0.973007,0.768778,0.768778,0.567513,0.694314,0.585691,0.874770,00:06:15.742752
KF_4,0.970212,0.791641,0.794963,0.560565,0.705945,0.571788,0.851223,00:05:24.004836
KF_5,0.970104,0.768344,0.768344,0.568724,0.691522,0.587794,0.878585,00:05:41.480344
KF_6,0.975099,0.771616,0.771616,0.566467,0.726495,0.580509,0.853986,00:07:27.608490
KF_7,0.973092,0.782438,0.782438,0.568100,0.717092,0.584744,0.863457,00:07:26.120357
KF_8,0.971212,0.753422,0.753422,0.573293,0.683283,0.594737,0.890292,00:06:24.817881
KF_9,0.975864,0.766110,0.769443,0.565357,0.705579,0.581087,0.864228,00:11:00.143461
KF_10,0.969073,0.772395,0.772395,0.574386,0.692472,0.596319,0.888173,00:06:12.086471


## CNN 2 MAX SM

In [11]:
# info
model_name = 'CNN 2 MAX SM'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)    
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(96, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97516 samples, validate on 13684 samples
Epoch 1/10000
97516/97516 [==============================] - 17s - loss: 0.5057 - acc: 0.7765 - val_loss: 0.4256 - val_acc: 0.8207
Epoch 2/10000
97516/97516 [==============================] - 15s - loss: 0.3701 - acc: 0.8456 - val_loss: 0.3720 - val_acc: 0.8240
Epoch 3/10000
97516/97516 [==============================] - 15s - loss: 0.3423 - acc: 0.8582 - val_loss: 0.4043 - val_acc: 0.8004
Epoch 4/10000
97516/97516 [==============================] - 15s - loss: 0.3266 - acc: 0.8660 - val_loss: 0.3478 - val_acc: 0.8357
Epoch 5/10000
97516/97516 [==============================] - 15s - loss: 0.3155 - acc: 0.8705 - val_loss: 0.4625 - val_acc: 0.7735
Epoch 6/10000
97516/97516 [==============================] - 15s - loss: 0.3089 - acc: 0.8734 - val_loss: 0.4267 - val_acc: 0.8082
Epoch 7/10000
97516/97516 [==============================] - 15s - loss: 0.3009 - acc: 0.8778 - val_loss: 0.3180 - val_acc: 0.8557
Epoch 8/10000
97516/97516 [=

97540/97540 [==============================] - 15s - loss: 0.5062 - acc: 0.7728 - val_loss: 1.4240 - val_acc: 0.4109
Epoch 2/10000
97540/97540 [==============================] - 14s - loss: 0.3778 - acc: 0.8438 - val_loss: 1.1191 - val_acc: 0.5325
Epoch 3/10000
97540/97540 [==============================] - 13s - loss: 0.3467 - acc: 0.8563 - val_loss: 0.2259 - val_acc: 0.8937
Epoch 4/10000
97540/97540 [==============================] - 13s - loss: 0.3315 - acc: 0.8635 - val_loss: 2.3456 - val_acc: 0.2953
Epoch 5/10000
97540/97540 [==============================] - 13s - loss: 0.3304 - acc: 0.8638 - val_loss: 0.3122 - val_acc: 0.8658
Epoch 6/10000
97540/97540 [==============================] - 13s - loss: 0.3139 - acc: 0.8705 - val_loss: 0.5301 - val_acc: 0.7421
Epoch 7/10000
97540/97540 [==============================] - 13s - loss: 0.3049 - acc: 0.8745 - val_loss: 0.4162 - val_acc: 0.8123
Epoch 8/10000
97540/97540 [==============================] - 13s - loss: 0.2985 - acc: 0.8773 - v

97557/97557 [==============================] - 14s - loss: 0.5279 - acc: 0.7593 - val_loss: 0.7687 - val_acc: 0.5709
Epoch 2/10000
97557/97557 [==============================] - 13s - loss: 0.4122 - acc: 0.8156 - val_loss: 0.4979 - val_acc: 0.7647
Epoch 3/10000
97557/97557 [==============================] - 13s - loss: 0.3559 - acc: 0.8508 - val_loss: 0.4874 - val_acc: 0.7900
Epoch 4/10000
97557/97557 [==============================] - 13s - loss: 0.3390 - acc: 0.8596 - val_loss: 0.4066 - val_acc: 0.7967
Epoch 5/10000
97557/97557 [==============================] - 13s - loss: 0.3260 - acc: 0.8634 - val_loss: 0.4644 - val_acc: 0.8052
Epoch 6/10000
97557/97557 [==============================] - 13s - loss: 0.3168 - acc: 0.8684 - val_loss: 0.3978 - val_acc: 0.7979
Epoch 7/10000
97557/97557 [==============================] - 13s - loss: 0.3076 - acc: 0.8733 - val_loss: 0.4222 - val_acc: 0.8069
Epoch 8/10000
97557/97557 [==============================] - 13s - loss: 0.3007 - acc: 0.8768 - v

In [12]:
print(model_name)

metrics_kf

CNN 2 MAX SM


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.966007,0.783253,0.783253,0.554751,0.693472,0.561750,0.842562,00:07:46.847954
KF_2,0.960512,0.796974,0.796974,0.573649,0.740785,0.592344,0.863080,00:05:29.835648
KF_3,0.966809,0.772741,0.772741,0.558028,0.706545,0.566368,0.843199,00:06:58.618028
KF_4,0.963814,0.789240,0.789240,0.553295,0.732889,0.547590,0.802026,00:07:44.660877
KF_5,0.964866,0.788255,0.788255,0.564033,0.701097,0.579093,0.863325,00:06:56.367117
KF_6,0.965248,0.789961,0.789961,0.566347,0.720312,0.581104,0.857406,00:05:47.078294
KF_7,0.967631,0.782432,0.782432,0.564060,0.717984,0.576969,0.852933,00:05:56.560207
KF_8,0.969844,0.766949,0.766949,0.560273,0.671127,0.575398,0.873060,00:07:55.121352
KF_9,0.965310,0.794306,0.794306,0.563436,0.706173,0.577467,0.859229,00:07:33.029763
KF_10,0.963765,0.783461,0.783461,0.555795,0.708024,0.561168,0.835153,00:05:45.284558


## CNN 3 MAX SM

In [13]:
# info
model_name = 'CNN 3 MAX SM'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)

    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(24, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97618 samples, validate on 13684 samples
Epoch 1/10000
97618/97618 [==============================] - 14s - loss: 0.5960 - acc: 0.7198 - val_loss: 0.4881 - val_acc: 0.7274
Epoch 2/10000
97618/97618 [==============================] - 14s - loss: 0.5383 - acc: 0.7550 - val_loss: 0.7161 - val_acc: 0.6071
Epoch 3/10000
97618/97618 [==============================] - 14s - loss: 0.5031 - acc: 0.7619 - val_loss: 0.4828 - val_acc: 0.7064
Epoch 4/10000
97618/97618 [==============================] - 14s - loss: 0.4489 - acc: 0.8070 - val_loss: 0.4992 - val_acc: 0.8202
Epoch 5/10000
97618/97618 [==============================] - 14s - loss: 0.4322 - acc: 0.8201 - val_loss: 0.3646 - val_acc: 0.8625
Epoch 6/10000
97618/97618 [==============================] - 14s - loss: 0.4256 - acc: 0.8221 - val_loss: 0.3219 - val_acc: 0.8660
Epoch 7/10000
97618/97618 [==============================] - 14s - loss: 0.4167 - acc: 0.8238 - val_loss: 0.4195 - val_acc: 0.8516
Epoch 8/10000
97618/97618 [=

97535/97535 [==============================] - 14s - loss: 0.4216 - acc: 0.8116 - val_loss: 0.4992 - val_acc: 0.6942
Epoch 27/10000
97535/97535 [==============================] - 14s - loss: 0.4204 - acc: 0.8120 - val_loss: 0.4365 - val_acc: 0.7048
Epoch 28/10000
97535/97535 [==============================] - 14s - loss: 0.4213 - acc: 0.8114 - val_loss: 0.4749 - val_acc: 0.6832
Epoch 29/10000
97535/97535 [==============================] - 14s - loss: 0.4140 - acc: 0.8122 - val_loss: 0.5017 - val_acc: 0.6611
Epoch 30/10000
97535/97535 [==============================] - 14s - loss: 0.4041 - acc: 0.8134 - val_loss: 0.4177 - val_acc: 0.7179
Epoch 31/10000
97535/97535 [==============================] - 14s - loss: 0.4011 - acc: 0.8156 - val_loss: 0.4415 - val_acc: 0.7132
Epoch 32/10000
97535/97535 [==============================] - 14s - loss: 0.4002 - acc: 0.8160 - val_loss: 0.4127 - val_acc: 0.7193
Epoch 33/10000
97535/97535 [==============================] - 14s - loss: 0.3988 - acc: 0.8

97540/97540 [==============================] - 14s - loss: 0.3868 - acc: 0.8349 - val_loss: 0.3107 - val_acc: 0.8782
Epoch 17/10000
97540/97540 [==============================] - 14s - loss: 0.3882 - acc: 0.8361 - val_loss: 0.6549 - val_acc: 0.7495
Epoch 18/10000
97540/97540 [==============================] - 14s - loss: 0.3826 - acc: 0.8400 - val_loss: 0.5524 - val_acc: 0.7670
Epoch 19/10000
97540/97540 [==============================] - 14s - loss: 0.3816 - acc: 0.8377 - val_loss: 0.5692 - val_acc: 0.8025
Epoch 20/10000
97540/97540 [==============================] - 14s - loss: 0.3817 - acc: 0.8393 - val_loss: 1.6265 - val_acc: 0.5271
Epoch 21/10000
97540/97540 [==============================] - 14s - loss: 0.3921 - acc: 0.8312 - val_loss: 1.5036 - val_acc: 0.4754
Epoch 22/10000
97540/97540 [==============================] - 14s - loss: 0.3841 - acc: 0.8372 - val_loss: 0.7390 - val_acc: 0.7233
Epoch 23/10000
97540/97540 [==============================] - 14s - loss: 0.3798 - acc: 0.8

97603/97603 [==============================] - 14s - loss: 0.3348 - acc: 0.8671 - val_loss: 0.3254 - val_acc: 0.8495
Epoch 67/10000
97603/97603 [==============================] - 14s - loss: 0.3354 - acc: 0.8678 - val_loss: 0.3229 - val_acc: 0.8519
Epoch 68/10000
97603/97603 [==============================] - 14s - loss: 0.3322 - acc: 0.8681 - val_loss: 0.3111 - val_acc: 0.8583
Epoch 69/10000
97603/97603 [==============================] - 14s - loss: 0.3332 - acc: 0.8676 - val_loss: 0.3681 - val_acc: 0.8302
Epoch 70/10000
97603/97603 [==============================] - 14s - loss: 0.3339 - acc: 0.8674 - val_loss: 0.4170 - val_acc: 0.8259
Epoch 71/10000
97603/97603 [==============================] - 14s - loss: 0.3346 - acc: 0.8672 - val_loss: 0.4519 - val_acc: 0.8275
Epoch 72/10000
97536/97603 [============================>.] - ETA: 0sKF_9
Train on 97502 samples, validate on 13684 samples
Epoch 1/10000
97502/97502 [==============================] - 14s - loss: 0.5814 - acc: 0.7217 - val

In [14]:
print(model_name)

metrics_kf

CNN 3 MAX SM


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.930188,0.794255,0.797577,0.556235,0.700956,0.563530,0.841641,00:06:23.413820
KF_2,0.926693,0.807058,0.807058,0.570612,0.729294,0.588018,0.862423,00:09:11.920597
KF_3,0.932147,0.778163,0.778163,0.569997,0.702551,0.589223,0.875296,00:06:51.904900
KF_4,0.929158,0.786022,0.786022,0.535289,0.694900,0.492423,0.713760,00:13:24.454142
KF_5,0.932870,0.782122,0.782122,0.565057,0.695942,0.581432,0.868719,00:11:18.474355
KF_6,0.939721,0.804351,0.804351,0.561727,0.726017,0.570759,0.840831,00:06:07.749598
KF_7,0.934791,0.795612,0.798935,0.556590,0.717216,0.560824,0.830045,00:17:11.665083
KF_8,0.942057,0.773922,0.773922,0.559824,0.692193,0.572343,0.858458,00:16:56.566000
KF_9,0.895671,0.742350,0.742350,0.535117,0.691526,0.494831,0.720563,00:06:25.467400
KF_10,0.900389,0.747846,0.751179,0.538458,0.716207,0.494315,0.709643,00:07:58.656408


# Metrics

In [15]:
models

,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
CNN 1 MAX RO,0.947721,0.733517,0.733849,0.552112,0.705972,0.551824,0.820640,00:08:12.992410
CNN 2 MAX RO,0.925944,0.758130,0.759128,0.547407,0.717398,0.533984,0.785031,00:07:33.695200
CNN 3 MAX RO,0.825688,0.784327,0.784992,0.538630,0.717383,0.485715,0.689475,00:10:29.193860
CNN 1 MAX SM,0.972182,0.772381,0.773711,0.567323,0.702224,0.584159,0.869166,00:06:39.310337
CNN 2 MAX SM,0.965381,0.784757,0.784757,0.561367,0.709841,0.571925,0.849197,00:06:47.340379
CNN 3 MAX SM,0.926368,0.781170,0.782168,0.554891,0.706680,0.550770,0.812138,00:10:11.027230
